### まずはkこの数字でvalを作れるかを判定する関数を作成

In [6]:
def solver(k, min_val, max_val, overlap, val):
    operators = ["+", "-", "*", "/"]
    results = []

    def dfs(expression, remaining):
        # ベースケース: 望ましい数の数字が使用されている場合
        if len(expression.split()) // 2 == k:
            # 式を評価し、目的の値と一致するかどうかを確認する
            if eval(expression) == val:
                results.append(expression)
            return 
        
        for num in range(min_val, max_val + 1):
            for op in operators:
                # 次の数字と演算子を式に追加
                new_expression = expression + ("" if expression == "" else " ") + op + " " + str(num)
                dfs(new_expression, remaining - 1)

    # 初期状態から開始
    for num in range(min_val, max_val + 1):
        dfs(str(num), k - 1)

    return results


In [7]:
# make10
output = solver(k=4, min_val=1, max_val=9, overlap=False, val=10)